# Dem Products


When working with a Digital Elevation Model (DEM), there are several outputs or derived products that can be generated based on the elevation data. Here are some common outputs obtained from DEM analysis:

**Hillshade:** Hillshade is a visualization technique that creates a shaded relief map to depict the terrain's three-dimensional characteristics. It uses lighting angles to simulate the effects of illumination on the surface, resulting in shadows and highlights that emphasize the terrain's shape and topography.

**Slope:** Slope represents the steepness of the terrain at each location. It is calculated as the rate of change in elevation over a given distance. Slope values are typically expressed in degrees or as a percentage. Slope analysis is useful for understanding the terrain's gradient and identifying areas with steep slopes.

**Aspect:** Aspect describes the direction in which a slope faces. It is usually measured in degrees, representing the azimuth angle from north (0°-360°) toward which the slope faces. Aspect analysis provides information about the orientation of the terrain and its exposure to different solar radiation or wind patterns.

**Contour Lines:** Contour lines are lines that connect points of equal elevation on a terrain surface. They are widely used in cartography to represent the shape and relief of the land. Contour lines provide a visual representation of elevation changes, and their spacing can indicate the steepness of the terrain.

**Watershed Delineation:** Using DEM data, watershed delineation techniques can be applied to identify the drainage basins or catchment areas of rivers and streams. Watershed delineation is important for hydrological studies, water resource management, and understanding the flow patterns of surface water.

**Viewshed Analysis:** Viewshed analysis determines the areas visible or hidden from a given viewpoint in the terrain. It helps identify locations with good visibility or line of sight, making it useful for siting observation points, telecommunication towers, or surveillance systems.

**Terrain Classification:** DEM data can be used for classifying terrain features, such as flat areas, hills, valleys, ridges, and cliffs. Classification can be based on slope, curvature, or other terrain attributes, enabling the identification and characterization of different landforms.

These are some common outputs that can be derived from DEM analysis. Depending on the specific requirements of your project or analysis, other derived products such as roughness, aspect-based solar radiation, or elevation profiles may also be generated.

In [ ]:
# Hillshade
import numpy as np
import rasterio

# Read the DEM file
dem_path = 'path/to/dem.tif'
with rasterio.open(dem_path) as src:
    dem = src.read(1)  # Read the first band of the DEM
    transform = src.transform  # Get the affine transformation matrix

# Calculate hillshade
azimuth = 315  # Angle of the light source (in degrees)
altitude = 45  # Angle of the light source above the horizon (in degrees)
hillshade = np.around(src.hillshade(dem, azimuth, altitude), decimals=2)

# Create a new raster file for the hillshade
hillshade_path = 'path/to/hillshade.tif'
with rasterio.open(hillshade_path, 'w', driver='GTiff', height=hillshade.shape[0], width=hillshade.shape[1], count=1, dtype=hillshade.dtype, crs=src.crs, transform=transform) as dst:
    dst.write(hillshade, 1)  # Write the hillshade data to the raster file

In [ ]:
# Slope and Aspect
import numpy as np
import rasterio

# Read the DEM file
dem_path = 'path/to/dem.tif'
with rasterio.open(dem_path) as src:
    dem = src.read(1)  # Read the first band of the DEM
    transform = src.transform  # Get the affine transformation matrix

# Calculate slope
x, y = np.gradient(dem, transform[0], transform[4])
slope = np.arctan(np.sqrt(x**2 + y**2)) * (180 / np.pi)

# Calculate aspect
aspect = np.arctan2(-x, y) * (180 / np.pi)
aspect[aspect < 0] += 360  # Adjust aspect values from -180 to 180 to 0 to 360 range

# Create a new raster file for slope
slope_path = 'path/to/slope.tif'
with rasterio.open(slope_path, 'w', driver='GTiff', height=slope.shape[0], width=slope.shape[1], count=1, dtype=slope.dtype, crs=src.crs, transform=transform) as dst:
    dst.write(slope, 1)  # Write the slope data to the raster file

# Create a new raster file for aspect
aspect_path = 'path/to/aspect.tif'
with rasterio.open(aspect_path, 'w', driver='GTiff', height=aspect.shape[0], width=aspect.shape[1], count=1, dtype=aspect.dtype, crs=src.crs, transform=transform) as dst:
    dst.write(aspect, 1)  # Write the aspect data to the raster file


In [ ]:
from osgeo import gdal, ogr, osr

# Generate contours by interval provided
def generate_contours_by_interval(dem, out_shp_path, contour_interval):
    band = dem.GetRasterBand(1)
    nodata = band.GetNoDataValue()
    proj = osr.SpatialReference(wkt=dem.GetProjection())

    # Generate the contour lines
    contours_ds = ogr.GetDriverByName('ESRI Shapefile').CreateDataSource(out_shp_path)
    contour_lyr = contours_ds.CreateLayer('contours', proj)
    contour_lyr.CreateField(ogr.FieldDefn('elevation', ogr.OFTReal))
    
    gdal.ContourGenerate(band, contour_interval, 0, [], 1, nodata, contour_lyr, 0, 0)

    # Clean up and return the results
    contours_ds = None
    del contours_ds

dem = gdal.Open('clipped_raster.tif')
generate_contours_by_interval(dem, 'contours.shp', 1)

In [ ]:
from osgeo import gdal, ogr, osr

# Generate contours by number of contours provided
def generate_contours_by_number(dem, out_shp_path, number_of_contours):
    band = dem.GetRasterBand(1)
    nodata = band.GetNoDataValue()
    proj = osr.SpatialReference(wkt=dem.GetProjection())

    # Get contour interval from min and max value
    (min, max, mean, std) = band.GetStatistics(True, True)
    contour_interval = (max - min) / number_of_contours

    # Generate the contour lines
    contours_ds = ogr.GetDriverByName('ESRI Shapefile').CreateDataSource(out_shp_path)
    contour_lyr = contours_ds.CreateLayer('contours', proj)
    contour_lyr.CreateField(ogr.FieldDefn('elevation', ogr.OFTReal))
    
    gdal.ContourGenerate(band, contour_interval, 0, [], 1, nodata, contour_lyr, 0, 0)

    # Clean up and return the results
    contours_ds = None
    del contours_ds

dem = gdal.Open('clipped_raster.tif')
generate_contours_by_number(dem, 'contours_num.shp', 10)

In [ ]:
# Watershed delineation using DEM data
import numpy as np
import rasterio
from scipy.ndimage import label, generate_binary_structure

# Read the DEM file
dem_path = 'path/to/dem.tif'
with rasterio.open(dem_path) as src:
    dem = src.read(1)  # Read the first band of the DEM
    transform = src.transform  # Get the affine transformation matrix

# Set a threshold to define the pouring points for the watersheds
pour_threshold = 1000  # Adjust this value as per your DEM and application

# Generate a binary mask of the pour points based on the threshold
pour_points = np.where(dem > pour_threshold, 1, 0)

# Apply watershed algorithm to delineate the watersheds
structure = generate_binary_structure(2, 1)  # Define the neighborhood connectivity
labels, num_watersheds = label(pour_points, structure)

# Create a new raster file for the watershed delineation
watershed_path = 'path/to/watershed.tif'
with rasterio.open(watershed_path, 'w', driver='GTiff', height=dem.shape[0], width=dem.shape[1], count=1, dtype=labels.dtype, crs=src.crs, transform=transform) as dst:
    dst.write(labels, 1)  # Write the watershed labels to the raster file

In [ ]:
# Viewshed analysis of a DEM
import numpy as np
import rasterio

def calculate_viewshed(dem, observer_coords):
    # Create an empty array to store the visibility values
    visibility = np.zeros_like(dem, dtype=np.int)

    # Iterate over each cell in the DEM
    for row in range(dem.shape[0]):
        for col in range(dem.shape[1]):
            # Skip if the cell is the observer's location
            if (row, col) == observer_coords:
                continue
            
            # Calculate the elevation difference between the observer and the cell
            delta_z = dem[observer_coords] - dem[row, col]

            # Calculate the horizontal distance between the observer and the cell
            distance = np.sqrt((row - observer_coords[0])**2 + (col - observer_coords[1])**2)

            # Calculate the angle of inclination from the observer to the cell
            angle = np.arctan2(delta_z, distance)

            # Check if the angle is within the observer's field of view
            if angle > np.radians(45):  # Adjust the angle threshold as needed
                visibility[row, col] = 1

    return visibility

# Read the DEM file
dem_path = 'path/to/dem.tif'
with rasterio.open(dem_path) as src:
    dem = src.read(1)  # Read the first band of the DEM
    transform = src.transform  # Get the affine transformation matrix

# Define the coordinates of the observer's location (row, col)
observer_coords = (50, 50)  # Adjust the coordinates as needed

# Perform viewshed analysis
visibility = calculate_viewshed(dem, observer_coords)

# Create a new raster file for the viewshed
viewshed_path = 'path/to/viewshed.tif'
with rasterio.open(viewshed_path, 'w', driver='GTiff', height=visibility.shape[0], width=visibility.shape[1], count=1, dtype=visibility.dtype, crs=src.crs, transform=transform) as dst:
    dst.write(visibility, 1)  # Write the viewshed data to the raster file

In [ ]:
# Terrain Classification using K-means clustering
import numpy as np
import rasterio
from sklearn.cluster import KMeans

# Read the DEM file
dem_path = 'path/to/dem.tif'
with rasterio.open(dem_path) as src:
    dem = src.read(1)  # Read the first band of the DEM
    transform = src.transform  # Get the affine transformation matrix

# Flatten the DEM into a 1D array
dem_flat = dem.flatten().reshape(-1, 1)

# Perform K-means clustering
n_clusters = 5  # Number of terrain classes to identify
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(dem_flat)

# Reshape the clustered labels to match the DEM shape
terrain_classes = kmeans.labels_.reshape(dem.shape)

# Create a new raster file for the terrain classification
terrain_path = 'path/to/terrain_classification.tif'
with rasterio.open(terrain_path, 'w', driver='GTiff', height=terrain_classes.shape[0], width=terrain_classes.shape[1], count=1, dtype=terrain_classes.dtype, crs=src.crs, transform=transform) as dst:
    dst.write(terrain_classes, 1)  # Write the terrain classification data to the raster file